In [1]:
!python --version

Python 3.13.2


In [4]:
import re
import os
import zipfile
import gc

import modin.pandas as pd
import numpy as np

import modin.config as modin_config
modin_config.Engine.put("dask")
modin_config.StorageFormat.put("pandas")

In [64]:
# Archive of World Flora Online (WFO) - https://zenodo.org/records/14538251
# families_dwc.tar.gz file contains zipped archives of classification data for all families
# each family is contained within a separate .zip file and the core data are in a file called `classification.csv'
# harvest the following columns from each of  these csv files
# family, genus, specificEpithet & majorGroup

In [5]:
wfo = pd.DataFrame(columns=["family", "genus", "specificEpithet", "majorGroup"], dtype=str)
wfo

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


,family,genus,specificEpithet,majorGroup


In [13]:
os.getcwd()

'C:\\Users\\90963425\\OneDrive - Western Sydney University\\PhD\\code'

In [30]:
for archive in os.listdir(r"./Families/"):
    with zipfile.ZipFile(file=f"./Families/{archive}", mode="r") as zf:
        with zf.open("classification.csv") as csf:
            df = pd.read_csv(filepath_or_buffer=csf, low_memory=False, encoding="latin")
            wfo = pd.concat([wfo, df.loc[:, ["family", "genus", "specificEpithet", "majorGroup"]]], ignore_index=True)

In [35]:
wfo.shape

(1780273, 4)

In [36]:
# A stands for Angiosperms???
wfo.loc[:, "majorGroup"].unique()

array(['A', 'Marchantiophyta', 'Bryophyta', 'Polypodiophyta',
       'Anthocerotophyta', 'Pinophyta', 'Cycadophyta', 'Ginkgophyta',
       'Lycopodiophyta'], dtype=object)

In [39]:
wfo.memory_usage()

Index                   132
family             14242184
genus              14242184
specificEpithet    14242184
majorGroup         14242184
dtype: int64

In [42]:
wfo_noduplicates = wfo.drop_duplicates()
wfo_noduplicates.shape

(1149343, 4)

In [49]:
wfo_noduplicates.isna().sum()

family                13
genus                726
specificEpithet    43254
majorGroup             0
dtype: int64

In [50]:
# the most important rows are family and genus 
wfo_noduplicates = wfo_noduplicates.dropna(subset=["family", "genus"])

In [51]:
wfo_noduplicates.isna().sum()

family                 0
genus                  0
specificEpithet    42520
majorGroup             0
dtype: int64

In [40]:
# wfo_noduplicates.to_csv(r"./wfo_taxonomy_clean.csv", index=False)

In [9]:
wfo_noduplicates = pd.read_csv(r"./wfo_taxonomy_clean.csv", low_memory=False)

In [2]:
groot = pd.read_csv(r"./GRooTFullVersion.csv", low_memory=False, encoding="latin")

Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


In [3]:
groot_taxonomy = groot.loc[:, ["family", "genus", "species"]].drop_duplicates().reset_index(drop=True)
groot_taxonomy.isna().sum()

family     6135
genus         0
species     138
dtype: int64

In [12]:
# 1959 unique genera in GRoot
groot_taxonomy.genus.drop_duplicates().size

1959

In [13]:
# we don't have the family info for all genera :(
np.intersect1d(groot_taxonomy.genus.drop_duplicates(), wfo_noduplicates.genus).size

1932

In [14]:
# this is because of case differences!
groot_taxonomy.genus.drop_duplicates().size - np.intersect1d(groot_taxonomy.genus.drop_duplicates(), wfo_noduplicates.genus).size

27

In [15]:
# we still have some work to do manually
missing = set(groot_taxonomy.genus.str.title().str.strip()).difference(set(wfo_noduplicates.genus.str.strip().str.title()))
missing

{'Anona',
 'Braciaria',
 'Cyanodon',
 'Helychrisum',
 'Matthiolaria',
 'Rhipogonum',
 'Rynchosia'}

In [16]:
# these do contain "anona", but.......
wfo_noduplicates.genus[wfo_noduplicates.genus.str.contains("anona")]

45827     Diclinanona
45828     Diclinanona
45829     Diclinanona
45830     Diclinanona
50994       Stenanona
50996       Stenanona
50997       Stenanona
50998       Stenanona
50999       Stenanona
51000       Stenanona
51001       Stenanona
51002       Stenanona
51006       Stenanona
51007       Stenanona
51008       Stenanona
51009       Stenanona
51015       Stenanona
51018       Stenanona
51028       Stenanona
51029       Stenanona
51030       Stenanona
51031       Stenanona
324046    Canonanthus
324053    Canonanthus
Name: genus, dtype: object

In [17]:
# welp
[wfo_noduplicates.genus[wfo_noduplicates.genus.str.startswith(g)].size for g in missing]

[0, 0, 0, 0, 0, 0, 0]

In [18]:
groot_taxonomy.genus.str.title().str.contains("Anona").sum()

np.int64(1)

In [19]:
# seems like Anona is legitimately missing
wfo_noduplicates.loc[wfo_noduplicates.family.str.contains("Annonaceae"), "genus"].unique()

array(['Cananga', 'Cyathocalyx', 'Drepananthus', 'Lettowianthus',
       'Meiocarpidium', 'Ambavia', 'Cleistopholis', 'Mezzettia',
       'Tetrameranthus', 'Anaxagorea', 'Annona', 'Asimina', 'Diclinanona',
       'Disepalum', 'Goniothalamus', 'Anonidium', 'Neostenanthera',
       'Bocagea', 'Hornschuchia', 'Trigynaea', 'Cardiopetalum',
       'Cymbopetalum', 'Froesiodendron', 'Porcelia', 'Mkilua',
       'Artabotrys', 'Letestudoxa', 'Pseudartabotrys', 'Duckeanthus',
       'Duguetia', 'Fusaea', 'Guatteria', 'Asteranthe', 'Hexalobus',
       'Uvariastrum', 'Isolona', 'Monodora', 'Dennettia', 'Lukea',
       'Mischogyne', 'Monocyclanthus', 'Uvariodendron', 'Uvariopsis',
       'Ophrypetalum', 'Sanrafaelia', 'Afroguatteria', 'Cleistochlamys',
       'Sphaerocoryne', 'Toussaintia', 'Dasymaschalon', 'Desmos',
       'Friesodielsia', 'Monanthotaxis', 'Fissistigma', 'Pyramidanthe',
       'Dielsiothamnus', 'Uvaria', 'Xylopia', 'Annickia',
       'Dendrokingstonia', 'Fenerivia', 'Maasia', 'Cre

In [20]:
groot_taxonomy.genus[groot_taxonomy.genus.str.startswith("Mat")]

766       Matteuccia
2913      Matteuccia
3544       Matthiola
4061      Matricaria
4062      Matteuccia
4555      Matricaria
4911       Matthiola
4912       Matthiola
5296      Matricaria
5436         Matayba
5496    Matthiolaria
5521       Matthiola
Name: genus, dtype: object

In [21]:
# looks like GRoot has some spelling mistakes in the genus names!!! 
# YIKES

groot_spelling_corrections = {
"Anona" : "Annona",
"Braciaria" : "Brachiaria",
"Cyanodon" : "Cynodon",
"Helychrisum" : "Helichrysum",
"Matthiolaria" : "Matthiola", # https://list-ui-wfo-staging.rbge.info/taxon/wfo-0000368988-2018-07?page=1 - synonyms
"Rhipogonum" : "Ripogonum",
"Rynchosia" : "Rhynchosia"
}

In [22]:
spelling_corrected_groot_genera = groot_taxonomy.genus.\
        apply(func= lambda g: groot_spelling_corrections.get(g) if g in groot_spelling_corrections.keys() else g.strip().title())

In [23]:
spelling_corrected_groot_genera[spelling_corrected_groot_genera.str.startswith("Ann")]

1167    Annona
2444    Annona
Name: genus, dtype: object

In [10]:
set(spelling_corrected_groot_genera).difference(set(wfo_noduplicates.genus.str.strip().str.title()))

set()

In [11]:
set(spelling_corrected_groot_genera.str.title()).difference(set(wfo_noduplicates.genus.str.strip().str.title()))

set()

In [12]:
# finally!
print(set(spelling_corrected_groot_genera.str.title().str.strip()).difference(set(wfo_noduplicates.genus.str.strip().str.title())))

# the problem is GRoot NOT the WFO database!
print(set(spelling_corrected_groot_genera.str.title().str.strip()).difference(set(wfo_noduplicates.genus)))

set()
set()


In [13]:
spelling_corrected_groot_genera #.reset_index(drop=True)

0          Betula
1           Picea
2           Pinus
3       Agropyron
4       Artemisia
          ...    
7229    Valeriana
7230     Veronica
7231        Vicia
7232        Vicia
7233    Centaurea
Name: genus, Length: 7234, dtype: object

In [24]:
groot_taxonomy

,family,genus,species
0,NaN,Betula,NaN
1,NaN,Picea,NaN
2,NaN,Pinus,NaN
3,NaN,Agropyron,cristatum
4,NaN,Artemisia,tridentata
...,...,...,...
7229,NaN,Valeriana,excelsa
7230,NaN,Veronica,aphylla
7231,NaN,Vicia,onobrychioides
7232,NaN,Vicia,oroboides


In [28]:
# mhmmmm
groot_taxonomy.genus = spelling_corrected_groot_genera
groot_taxonomy.reset_index(inplace=True, drop=True)

In [29]:
groot_taxonomy

,family,genus,species
0,NaN,Betula,NaN
1,NaN,Picea,NaN
2,NaN,Pinus,NaN
3,NaN,Agropyron,cristatum
4,NaN,Artemisia,tridentata
...,...,...,...
7229,NaN,Valeriana,excelsa
7230,NaN,Veronica,aphylla
7231,NaN,Vicia,onobrychioides
7232,NaN,Vicia,oroboides


In [30]:
# groot_taxonomy.dtypes
groot_taxonomy.family.dtype

dtype('O')

In [31]:
temp = wfo_noduplicates.loc[:, ["family", "genus"]].drop_duplicates().reset_index(drop=True)
families = pd.Series(index=temp.genus.values, data=temp.family.values, dtype=str, copy=True)

In [32]:
temp.isna().sum()

family    0
genus     0
dtype: int64

In [37]:
families

Acanthopale          Acanthaceae
Acanthopsis          Acanthaceae
Acanthus             Acanthaceae
Afrofittonia         Acanthaceae
Ambongia             Acanthaceae
                       ...      
Bisluederitzia    Zygophyllaceae
Guaiacon          Zygophyllaceae
Nitrapia          Zygophyllaceae
Quaiacum          Zygophyllaceae
Seezenia          Zygophyllaceae
Length: 50936, dtype: object

In [34]:
# :( ???
families["Cannabis"]

Cannabis    Cannabaceae
Cannabis     Urticaceae
Name: __reduced__, dtype: object

In [44]:
# GOT SO MANY CONFLICTS BETWEEN GROOT AND THE WFO DATABASE

n_conflicts : int = 0

for i in range(groot_taxonomy.shape[0]):
    if pd.isnull(groot_taxonomy.loc[i, "family"]): # if GRooT has NaN for family field in a record
        if len(families.get(groot_taxonomy.loc[i, "genus"], False)) != 1: # if WFO has more than one family name for a given genus, 
            raise ValueError(f"{groot_taxonomy.loc[i, "genus"]} has more than 1 family names in WFO database! {families.get(groot_taxonomy.loc[i, "genus"]).values}")
        groot_taxonomy.loc[i, "family"] = families.get(groot_taxonomy.loc[i, "genus"]) # replace the NaN with the family name from WFO database
    else: # if GRooT has a family name, check for conflicts with WFO
        if groot_taxonomy.loc[i, "family"] != families.get(groot_taxonomy.loc[i, "genus"]): # if there's a conflict
                print(f"{groot_taxonomy.loc[i, "genus"]:10s} Expected {families.get(groot_taxonomy.loc[i, "genus"])},\
                        got {groot_taxonomy.loc[i, "family"]}")
                n_conflicts += 1 # WELP :(
            

ValueError: Betula has more than 1 family names in WFO database! ['Betulaceae' 'Nothofagaceae']

7234

In [68]:
# 1.28 GiBs!!!!
os.path.getsize(r"./../classification.csv") / np.power(1024, 3)

np.float64(1.2805062485858798)

In [69]:
# let's see the csv (pre)compiled by WFO itself

wfo_full = pd.read_csv(r"./../classification.csv", low_memory=False, encoding="latin")

In [70]:
wfo_full.columns

Index(['taxonID', 'scientificNameID', 'localID', 'scientificName', 'taxonRank',
       'parentNameUsageID', 'scientificNameAuthorship', 'family', 'subfamily',
       'tribe', 'subtribe', 'genus', 'subgenus', 'specificEpithet',
       'infraspecificEpithet', 'verbatimTaxonRank', 'nomenclaturalStatus',
       'namePublishedIn', 'taxonomicStatus', 'acceptedNameUsageID',
       'originalNameUsageID', 'nameAccordingToID', 'taxonRemarks', 'created',
       'modified', 'references', 'source', 'majorGroup', 'tplID',
       'speciesHybridMarker', 'infraspecificRank', 'originalID', 'old_t1id',
       'tropicosId', 'references1.0', 'doNotProcess', 'doNotProcess_reason',
       'OfficialFamily', 'comments', 'deprecated'],
      dtype='object')

In [71]:
wfo_full.shape

(1704312, 40)

In [72]:
wfo_full.loc[:, ["family", "genus", "specificEpithet"]].drop_duplicates().reset_index(drop=True)

,family,genus,specificEpithet
0,Cyperaceae,Schoenoxiphium,ecklonii
1,Cyperaceae,Cyperus,violifolia
2,Cyperaceae,Carex,viridula
3,Cyperaceae,Mariscus,phleoides
4,Cyperaceae,Tetraria,compar
...,...,...,...
1161993,Brassicaceae,Cardamine,caesiella
1161994,Brassicaceae,Cardamine,chlorina
1161995,Brassicaceae,Sinalliaria,grandifolia
1161996,Brassicaceae,Ceratocnemum,mitabile


In [73]:
wfo_full.loc[:, ["family", "genus"]].isna().sum()

family    19093
genus      5622
dtype: int64

In [74]:
np.set_printoptions(threshold=np.inf)

# suppressed the outputs
np.unique(wfo_full.loc[:, "genus"].dropna(), return_counts=True);

In [76]:
wfo_full.loc[:, ["family", "genus"]].dropna(how="any").drop_duplicates()

,family,genus
0,Cyperaceae,Schoenoxiphium
1,Cyperaceae,Cyperus
2,Cyperaceae,Carex
3,Cyperaceae,Mariscus
4,Cyperaceae,Tetraria
...,...,...
1703701,Dicranellaceae,Fuscina
1703702,Ditrichaceae,Fuscina
1703750,Orthotrichaceae,Phragmidiolum
1703831,Brachytheciaceae,Leptodon


In [79]:
temp_wfo_full = wfo_full.loc[:, ["family", "genus"]].dropna(how="any").drop_duplicates().reset_index(drop=True)
wfo_full = pd.Series(index=temp_wfo_full.genus.values, data=temp_wfo_full.family.values)

In [82]:
wfo_full

Schoenoxiphium          Cyperaceae
Cyperus                 Cyperaceae
Carex                   Cyperaceae
Mariscus                Cyperaceae
Tetraria                Cyperaceae
                        ...       
Fuscina             Dicranellaceae
Fuscina               Ditrichaceae
Phragmidiolum      Orthotrichaceae
Leptodon          Brachytheciaceae
Catenularia          Polypodiaceae
Length: 46898, dtype: object

In [81]:
# 
wfo_full["Cynodon"]

Cynodon            Poaceae
Cynodon           Bryaceae
Cynodon    Flexitrichaceae
Cynodon      Distichiaceae
Cynodon         Pottiaceae
Cynodon     Dicranellaceae
Name: __reduced__, dtype: object

In [83]:
wfo_full["Pinus"]

Pinus      Araucariaceae
Pinus           Pinaceae
Pinus       Cupressaceae
Pinus    Sciadopityaceae
Pinus      Amaranthaceae
Name: __reduced__, dtype: object

In [85]:
wfo_full["Cyperus"]

Cyperus    Cyperaceae
Cyperus     Juncaceae
Name: __reduced__, dtype: object

In [86]:
wfo_full["Oryza"]

'Poaceae'

In [60]:
# using the WFO backbone dataset
# IT'S IDENTICAL TO CLASSIFICATION.CSV :(
# it is actually a tab delimited file with a .csv extension????

backbone = pd.read_table(r"\\ad.uws.edu.au\dfshare\HomesBNK$\90963425\Downloads\WFO_Backbone\classification.csv",
                         low_memory=False, encoding="latin", sep="\t")

In [62]:
backbone.columns

Index(['taxonID', 'scientificNameID', 'localID', 'scientificName', 'taxonRank',
       'parentNameUsageID', 'scientificNameAuthorship', 'family', 'subfamily',
       'tribe', 'subtribe', 'genus', 'subgenus', 'specificEpithet',
       'infraspecificEpithet', 'verbatimTaxonRank', 'nomenclaturalStatus',
       'namePublishedIn', 'taxonomicStatus', 'acceptedNameUsageID',
       'originalNameUsageID', 'nameAccordingToID', 'taxonRemarks', 'created',
       'modified', 'references', 'source', 'majorGroup', 'tplID'],
      dtype='object')

In [63]:
backbone.loc[:, ["family", "genus", "specificEpithet"]].isna().sum()

family             10664
genus               3038
specificEpithet    52477
dtype: int64

In [64]:
backbone.loc[:, ["family", "genus"]].isna().sum()

family    10664
genus      3038
dtype: int64

In [65]:
bbone = backbone.loc[:, ["family", "genus"]].drop_duplicates().dropna().reset_index(drop=True)
bbone = pd.Series(index=bbone.genus.values, data=bbone.family.values)

In [66]:
bbone["Cynodon"]

Cynodon            Poaceae
Cynodon           Bryaceae
Cynodon    Flexitrichaceae
Cynodon      Distichiaceae
Cynodon         Pottiaceae
Cynodon     Dicranellaceae
Name: __reduced__, dtype: object

In [61]:
# gc.collect()